### Imports

In [1]:
import os
import re
import tqdm

import matplotlib.pyplot as plt
import numpy as np

from sklearn.preprocessing import MinMaxScaler
from sklearn import svm
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import mean_absolute_error as mae

### Functions related to the model
(i.e. k-fold cv, running the model, metrics)

In [2]:
def cs(gt, pred, alpha):
    """
    Computes the Cumulative Score with given alpha

    :param gt:    Ground truths
    :param pred:  Predicted dates
    :param alpha: Acceptable error range in years

    :return: The CS with given alpha
    """
    count = 0
    for i in range(len(gt)):
        absolute_error = abs(gt[i] - pred[i])
        if absolute_error <= alpha:
            count += 1

    return count / len(gt) * 100

In [3]:
def kfold_cv(model, features, labels, k, seed):
    """
    Performs stratisfied k-fold cross-validation.

    :param model:    The model to train
    :param features: Features to train and test the model with
    :param labels:   Labels of the features
    :param k:        Number of folds for the cross-validation
    :param seed:     Seed for splitting data into test/train sets

    :return: Mean and SD of MAE, CS with alpha = 25 and alpha = 0 years across all folds.
    """
    mae_k = []
    cs_k = []
    cs_1 = []
    # As the dataset is imbalanced --> stratified kfold + seed to get the same validation/train splits
    kf = StratifiedKFold(n_splits=k, shuffle=True, random_state=seed)

    for train_index, test_index in kf.split(features, labels):
        # Getting test and train sets
        train_features, test_features = features[train_index], features[test_index]
        train_labels, test_labels = [labels[i] for i in train_index], [labels[i] for i in test_index]

        # Training model and predicting dates on test data
        model.fit(train_features, train_labels)
        pred = model.predict(test_features)
        pred = [int(i) for i in pred]

        mae_k.append(mae(test_labels, pred))
        cs_k.append(cs(test_labels, pred, 25))
        cs_1.append(cs(test_labels, pred, 0))

    return np.mean(mae_k), np.std(mae_k), np.mean(cs_k), np.std(cs_k), np.mean(cs_1), np.std(cs_1)

In [18]:
def kfold_cv_aug(model, features_norm, features_aug, labels_norm, labels_aug, k, n_aug, seed):
    """
    Performs stratisfied k-fold cross-validation for augmented data.

    :param model:         The model to train
    :param features_norm: Features of non-augmented images
    :param features_aug:  Features of augmented images
    :param labels_norm:   Labels of the non-augmented features
    :param labels_aug:    Labels of the augmented features
    :param k:             Number of folds for the cross-validation
    :param n_aug:         Number of augmented images per non-augmented image
    :param seed:          Seed for splitting data into test/train sets

    :return: Mean and SD of MAE, CS with alpha = 25 and alpha = 0 years across all folds.
    """
    mae_k = []
    cs_k = []
    cs_1 = []

    kf = StratifiedKFold(n_splits=k, shuffle=True, random_state=seed)
    for train_index, test_index in tqdm.tqdm(kf.split(features_norm, labels_norm)):
        # Getting test and train sets from non-augmented labels and features
        test = [features_norm[idx] for idx in test_index]
        test_labels = [labels_norm[idx] for idx in test_index]

        train = [features_norm[idx] for idx in train_index]
        train_labels = [labels_norm[idx] for idx in train_index]

        # Getting test and training sets from augmented labels and features
        for idx in range(len(features_aug)):
            if int(idx/n_aug) in test_index:
                continue
            else:
                train.append(features_aug[idx])
                train_labels.append(labels_aug[idx])

        # Training the model and predicting dates
        model.fit(train, train_labels)
        pred = model.predict(test)
        pred = [int(i) for i in pred]

        mae_k.append(mae(test_labels, pred))
        cs_k.append(cs(test_labels, pred, 25))
        cs_1.append(cs(test_labels, pred, 0))
    return np.mean(mae_k), np.std(mae_k), np.mean(cs_k), np.std(cs_k), np.mean(cs_1), np.std(cs_1)

In [5]:
def model(model, train_features, test_features, train_labels, test_labels):
    """
    Predicts dates with a given model.

    :param model:          The model to train
    :param train_features: Features to train the model witb
    :param test_features:  Features to test the model with
    :param train_labels:   Labels of train features
    :param test_labels:    Labels of test features

    :return: Mean Absolute Error (MAE) and Cumulative Score with alpha-values 25 and 0
    """
    model.fit(train_features, train_labels)
    pred = model.predict(test_features)
    pred = [int(i) for i in pred]
    return round(mae(test_labels, pred), 4), round(cs(test_labels, pred, 25), 4), round(cs(test_labels, pred, 1), 4)

def plot_model(model, train_features, test_features, train_labels, test_labels):
    """
    Predicts dates with a given model and displays a scatter plot of 
    the ground truth and predicted dates.

    :param model:          The model to train
    :param train_features: Features to train the model witb
    :param test_features:  Features to test the model with
    :param train_labels:   Labels of train features
    :param test_labels:    Labels of test features

    :return: Mean Absolute Error (MAE) and Cumulative Score with alpha-values 25 and 0
    """
    model.fit(train_features, train_labels)
    pred = model.predict(test_features)
    pred = [int(i) for i in pred]

    plt.scatter(test_labels, pred)
    plt.ylabel('pred')
    plt.xlabel('true')
    plt.show()
    return round(mae(test_labels, pred), 4), round(cs(test_labels, pred, 25), 4), round(cs(test_labels, pred, 1), 4)

### Functions to process the features

In [6]:
HINGE = 0
JUNC = 1

In [7]:
def rescale_features(features):
    """
    Rescales features between 0 and 1
    """
    features = np.asarray(features)
    scaler = MinMaxScaler()
    scaler.fit(features)
    data_rescaled = scaler.transform(features)

    return data_rescaled

In [8]:
def rescale_features_split(features1, features2):
    """
    Rescales features between 0 and 1 in the case of 2 split sets of features    
    """
    features1 = np.array(features1)
    features2 = np.array(features2)

    scaler = MinMaxScaler()
    scaler.fit(np.concatenate((features1, features2), axis=0))
    data_rescaled1 = scaler.transform(features1)
    data_rescaled2 = scaler.transform(features2)
    return data_rescaled1, data_rescaled2

In [13]:
def get_features2(feature_dir, hinge_or_junc, n_aug):
    """
    Reads feature vectors from files in a directory

    :param feature_dir:   Directory containing feature files
    :param hinge_or_junc: Boolean value whether feature is from Hinge family (0) 
                          or is the Junclets feature (1)
    :param n_aug:         Number of augmented images per non-augmented image

    :return: List of feature vectors and corresponding labels (key years)
    """
    features = []
    labels = []
    count = 0
    print(n_aug, feature_dir)

    # Iterating over all feature files
    for file in sorted(os.listdir(feature_dir)):
        aug_num = re.search("(_[0-9]?[0-9].p)", file)

        if aug_num is not None:
            aug_num = re.search("([0-9]?[0-9])", aug_num.group())
        if aug_num is None or (aug_num is not None and int(aug_num.group()) <= n_aug):
            if file != ".DS_Store" and not os.path.isdir(feature_dir + '/' + file):
                f = open(feature_dir + "/" + file)
                # Gets the file's feature vector
                for line in f.readlines():
                    line = line.rstrip().split(" ")
                    if hinge_or_junc == HINGE:
                        features.append(line[2:])
                    elif hinge_or_junc == JUNC:
                        features.append([float(el) for el in line])
                        count += 1

                    label = re.search("([0-9][0-9][0-9][0-9])", file) #MPS
                    #label = re.search("(-?[0-9][0-9][0-9])", file) #DSS
                    labels.append(int(label.group()))

    return features, labels

In [10]:
def get_features(featuredir, size, n_aug):
    """
    Returns features and labels in a directory
    """
    if featuredir != 'junclets' and featuredir != 'test/junclets':
        prefix = './Data/DSS/features/'   # Change with data set used
        features, labels = get_features2(prefix + featuredir, HINGE, n_aug)
    elif featuredir == 'junclets':
        prefix = './Data/DSS/features/' + featuredir + '/size_' + str(size) + '/'  # Change with data set used
        features, labels = get_features2(prefix, JUNC, n_aug)
    return features, labels

def get_features_aug(featuredir, size, n_aug):
    """
    Returns features and labels in a directory of both non-augmented and augmented images
    """
    if featuredir != 'junclets':
        prefix = './Data/DSS/features/' # Change with data set used
        features_norm, labels_norm = get_features2(prefix + featuredir, HINGE, n_aug)
        prefix = prefix + 'features_aug_15/' + featuredir + '_aug'          # Change with data set used
        features_aug, labels_aug = get_features2(prefix, HINGE, n_aug)
    else:
        prefix = './Data/DSS/features/junclets/size_' + str(size) + '/'      # Change with data set used
        features_norm, labels_norm = get_features2(prefix, JUNC, n_aug)
        prefix = './Data/DSS/features/junclets_aug/size_' + str(size) + '/'  # Change with data set used
        features_aug, labels_aug = get_features2(prefix, JUNC, n_aug)

    return features_norm, labels_norm, features_aug, labels_aug

### Getting indices for test set

In [13]:
# MPS
size_data = 3267
test_size = 0.1 #as a fraction
test_indices = np.random.choice(np.array([i for i in range(size_data)]), int(size_data * test_size), replace=False)
np.save('./test_indices.npy', test_indices)

In [ ]:
# EA and DSS
idx = 0
for file in sorted(os.listdir('./Data/DSS/DSS_jpg_re/')):
    label = re.search("(-?[0-9][0-9][0-9])", file)
    print(label.group(), idx)
    idx += 1

np.save('./Data/DSS/test_indices.npy', [0, 5, 6, 16, 17])

In [11]:
test_indices = np.load('./Data/DSS/test_indices.npy')
#test_indices = np.load('./tfsom/MPS/test_indices.npy')

## Tuning Hyperparameters
Uses K-fold cross validation with k = 10 for MPS and k = 4 for EA and DSS data

With original data

In [ ]:
# For Hinge features

feat_names = ['hinge', 'cohinge', 'quadhinge', 'deltahinge', 'tcchinge']

k = 4    # EA and DSS number of folds in cross-validation
# k = 10 # MPS number of folds in cross-validation
Cs = [pow(2, n) for n in range(-7, 10, 1)]  # hyper-parameter range of values
seeds = [0, 50, 100, 150, 200, 250]

# Iterates over all features
for featuredir in feat_names:
    print(featuredir)

    # Getting the feature vectors and labels
    features, labels = get_features(featuredir, 0, 0)
    features = rescale_features(features)

    # Getting the test set
    test = [features[idx] for idx in test_indices]
    test_labels = [labels[idx] for idx in test_indices]

    # Getting the train set
    train = []
    train_labels = []
    for idx in range(len(features)):
        if idx not in test_indices:
            train.append(features[idx])
            train_labels.append(labels[idx])

    results = [[0, 0, 0, 0, 0, 0, 0] for i in range(len(Cs))]

    # k-fold cross validation for all seeds and hyper-parameter values
    for seed in seeds:
        for c_idx in range(len(Cs)):
            results[c_idx][0] = Cs[c_idx]
            clf = kfold_cv(svm.SVC(kernel='linear', decision_function_shape='ovr', C=Cs[c_idx]), np.array(train), np.array(train_labels), k, seed)
            for res_idx in range(len(clf)):
                results[c_idx][res_idx + 1] += clf[res_idx]
    
    # Computing mean results across seeds
    for c_idx in range(len(results)):
        for res_idx in range(1, len(results[c_idx])):
            results[c_idx][res_idx] = results[c_idx][res_idx]/len(seeds)

    for res in results:
        print(res)
    np.save('./Data/DSS/validation_' + featuredir + '.npy', results) ## Change according to data set

In [ ]:
# For the Junclets feature

featuredir = 'junclets'

k = 4   # EA and DSS number of folds for cross-validation
# k = 10 # MPS number of folds for cross-validation
Cs = [pow(2, n) for n in range(-7,10, 1)]
seeds = [0, 50, 100, 150, 200, 250]

# Iterates over all sub-codebook sizes
for cb_size in range(5, 35, 5):
    print(cb_size)
    # Getting the feature vectors and labels
    features, labels = get_features(featuredir, cb_size, 0)
    features = rescale_features(features)

    # Getting the test set
    test = [features[idx] for idx in test_indices]
    test_labels = [labels[idx] for idx in test_indices]

    # Getting the train set
    train = []
    train_labels = []
    for idx in range(len(features)):
        if idx not in test_indices:
            train.append(features[idx])
            train_labels.append(labels[idx])

    results = [[0, 0, 0, 0, 0, 0, 0] for i in range(len(Cs))]
    # Cross-validation across all seeds and hyper-parameter values
    for seed in seeds:
        for c_idx in range(len(Cs)):
            results[c_idx][0] = Cs[c_idx]
            clf = kfold_cv(svm.SVC(kernel='linear', decision_function_shape='ovr', C=Cs[c_idx]), np.array(train), np.array(train_labels), k, seed)
            for res_idx in range(len(clf)):
                results[c_idx][res_idx + 1] += clf[res_idx]
    
    # Mean results across seeds
    for c_idx in range(len(results)):
        for res_idx in range(1, len(results[c_idx])):
            results[c_idx][res_idx] = results[c_idx][res_idx]/len(seeds)

    for res in results:
        print(res)
    np.save('./Data/DSS/validation_' + featuredir + '_' + str(cb_size) + '.npy', results) ## Change according to data set

With augmented data

In [ ]:
feat_names = ['hinge', 'cohinge', 'quadhinge', 'deltahinge', 'tcchinge', 'junclets']

## EA and DSS
# k = 4        # number of folds for cross-validation
# n_aug = 15   # number of augmented images per non-augmented image
# cb_size = 15 # size of codebook

## MPS
k = 10         # number of fold for cross-validation
n_aug = 3      # number of augmented image per non-augmented image
cb_size = 25   # size of codebook

Cs = [pow(2, n) for n in range(-7,10, 1)] # range of values for hyper-parameter
seeds = [0, 50, 100, 150, 200, 250]

# Iterates over all features
for featuredir in feat_names:
    print(featuredir)
    # Getting feature vectors and labels
    features_norm, labels_norm, features_aug, labels_aug = get_features_aug(featuredir, cb_size, n_aug)
    features_norm, features_aug = rescale_features_split(features_norm, features_aug)

    # Getting test set
    test = [features_norm[idx] for idx in test_indices]
    test_labels = [labels_norm[idx] for idx in test_indices]

    # Getting training set from the augmented feature vectors and labels
    train_aug = []
    train_aug_labels = []
    for idx in range(len(features_aug)):
        if int(idx/n_aug) in test_indices:
            continue
        else:
            train_aug.append(features_aug[idx])
            train_aug_labels.append(labels_aug[idx])

    # Getting training set from the non-augmented feature vectors and labels
    train_norm = []
    train_norm_labels = []
    for idx in range(len(features_norm)):
        if idx not in test_indices:
            train_norm.append(features_norm[idx])
            train_norm_labels.append(labels_norm[idx])
            
    results = [[0, 0, 0, 0, 0, 0, 0] for i in range(len(Cs))]

    # Cross validation across all seeds and hyper-parameter values
    for seed in tqdm.tqdm(seeds):
        print(seed)
        for c_idx in range(len(Cs)):
            print(Cs[c_idx])
            results[c_idx][0] = Cs[c_idx]
            clf = kfold_cv_aug(svm.SVC(kernel='linear', decision_function_shape='ovr', C=Cs[c_idx]), train_norm, train_aug, train_norm_labels, train_aug_labels, k, n_aug, seed=seed)
            for res_idx in range(len(clf)):
                results[c_idx][res_idx + 1] += clf[res_idx]
    
    # Mean results across seeds
    for c_idx in range(len(results)):
        for res_idx in range(1, len(results[c_idx])):
            results[c_idx][res_idx] = results[c_idx][res_idx]/len(seeds)

    for res in results:
        print(res)
    np.save('./Data/MPS/validation_' + featuredir + '_aug_' + '.npy', results) ## Change according to data set

# Testing

With original data

In [ ]:
feat_names = ['hinge', 'cohinge', 'quadhinge', 'deltahinge', 'tcchinge', 'junclets']

## EA and DSS
# k = 4                                  # number of folds for cross-validation
# n_aug = 15                             # number of augmented images per non-augmented image
# cb_size = 15                           # codebook size
# Cs = [1, 1, 1, 0.125, 0.125, 0.03125]  # hyper-parameters

## MPS
k = 10                                   # number of folds for cross-validation
n_aug = 3                                # number of augmented images per non-augmented image
cb_size = 25                             # codebook size
Cs = [8, 0.0625, 0.125, 1, 1, 0.0625]    # hyper-parameters

idx_c = 0


for featuredir in feat_names:
    print(featuredir)
    # Getting feature vectores + labels
    features, labels = get_features(featuredir, cb_size, 0)
    features = rescale_features(features)

    # Getting test set
    test = [features[idx] for idx in test_indices]
    test_labels = [labels[idx] for idx in test_indices]

    # Getting train set
    train = []
    train_labels = []
    for idx in range(len(features)):
        if idx not in test_indices:
            train.append(features[idx])
            train_labels.append(labels[idx])

    # Date prediction of test set
    print(idx_c, Cs[idx_c])
    mae_res, cs_res25, cs_res1 = plot_model(svm.SVC(kernel='linear', decision_function_shape='ovr', C=Cs[idx_c]), train, test, train_labels, test_labels)
    print("MAE: %.4f  \t CS (=25): %.4f  \t CS(=1): %.4f " % (mae_res, cs_res25, cs_res1))
    print("%.4f,%.4f,%.4f" % (mae_res, cs_res25, cs_res1))

    idx_c += 1

With augmented data

In [ ]:
feat_names = ['hinge', 'cohinge', 'quadhinge', 'deltahinge','tcchinge', 'junclets']

## EA and DSS
# k = 4                                  # number of folds for cross-validation
# n_aug = 15                             # number of augmented images per non-augmented image
# cb_size = 15                           # codebook size
# Cs = [1, 1, 1, 2, 0.25, 0.25]          # hyper-parameters

## MPS
k = 10                                   # number of folds for cross-validation
n_aug = 3                                # number of augmented images per non-augmented image
cb_size = 25                             # codebook size
Cs = [2, 0.0625, 0.0625, 1, 1, 0.0625]   # hyper-parameters

idx_c = 0


for featuredir in feat_names:
    print(featuredir)
    # Getting feature vectors + labels
    features_norm, labels_norm, features_aug, labels_aug = get_features_aug(featuredir, cb_size, n_aug)
    features_norm, features_aug = rescale_features_split(features_norm, features_aug)

    # Test set from non-augmented data
    test = [features_norm[idx] for idx in test_indices]
    test_labels = [labels_norm[idx] for idx in test_indices]
    
    # Train set augmented data
    train_aug = []
    train_aug_labels = []
    for idx in range(len(features_aug)):
        if int(idx/n_aug) in test_indices:
            continue
        else:
            train_aug.append(features_aug[idx])
            train_aug_labels.append(labels_aug[idx])

    # Train set non-augmented data
    train_norm = []
    train_norm_labels = []
    for idx in range(len(features_norm)):
        if idx not in test_indices:
            train_norm.append(features_norm[idx])
            train_norm_labels.append(labels_norm[idx])

    # concatenating augmented and non-augmented train sets
    train = train_norm + train_aug
    train_labels = train_norm_labels + train_aug_labels
    print(len(train), len(train_labels), len(train_aug))

    # Date prediction on test set
    mae_res, cs_res25, cs_res1 = plot_model(svm.SVC(kernel='linear', decision_function_shape='ovr', C=Cs[idx_c]), train, test, train_labels, test_labels)
    print("MAE: %.4f  \t CS (=25): %.4f  \t CS(=1): %.4f " % (mae_res, cs_res25, cs_res1))
    print("%.4f,%.4f,%.4f" % (mae_res, cs_res25, cs_res1))

    idx_c += 1

#### Other

In [1]:
# number of samples per key year MPS data set
aug = [i for i in sorted(os.listdir('./all/hinge/'))]
origin = [i for i in sorted(os.listdir('./all/hinge/'))]
print(len(origin))

test_labels = []
for idx in test_indices:
    label = re.search("([0-9][0-9][0-9][0-9])", origin[idx])
    test_labels.append(label.group())

for year in range(1300, 1575, 25):
    print(test_labels.count(str(year)))

FileNotFoundError: [Errno 2] No such file or directory: './all/hinge/'

In [ ]:
files = [i for i in sorted(os.listdir('./Data/DSS/features/cchinge/'))]

files.remove('.DS_Store')
print(len(test_indices))
for idx in test_indices:
    print(files[idx])
